In [9]:
import requests
import os
import json
import twitter_authentication as config
import time

# Save your bearer token in a file called twitter_authentication.py in this directory
bearer_token = config.bearer_token
query = '(#COVID) OR (#COVID-19)'
out_file = 'raw_tweets.json'

search_url = "https://api.twitter.com/2/tweets/search/all"

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {'query': query,
                'tweet.fields': 'author_id,public_metrics',
                'max_results': 500
               }


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers, params, next_token = None):
    if next_token:
        params['next_token'] = next_token
    response = requests.request("GET", search_url, headers=headers, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def get_tweets(num_tweets = 2000):
    tweets = []
    next_token = None
    while len(tweets) < num_tweets:
        headers = create_headers(bearer_token)
        json_response = connect_to_endpoint(search_url, headers, query_params, next_token)
        for tweet in json_response['data']:
            tweets.append(tweet)
        try:
            next_token = json_response['meta']['next_token']
        except KeyError:
            break
        time.sleep(2)
    return tweets



def main():
    tweets = get_tweets(500)
    with open(out_file, 'w') as f:
        json.dump(tweets, f)



if __name__ == "__main__":
    main()

200
200
200
200
200
RT @ShannonJoyRadio: This morning at 9am a single nurse who is refusing the #COVID jab at Highland Hospital STOOD UP to Goliath ... The Uni…
RT @MicroPapillon: ⚡🔞Un film #porno est diffusé par erreur en salle d'attente à l'hôpital de #Bayonne.

Personne ne s'est plaint de l'atten…
RT @JulikaBrand: in #Germany ➡️#police now attacks people, who address  #Covid-restrictions... https://t.co/RDKN8bGjfH
RT @laabdon: 🧪 Todos los hisopado de #COVID-19 realizados al plantel del Club @Nacional de Football dieron resultado #NEGATIVO. https://t.c…
RT @LeahCMa: @DubreuilhMarcel 🤣🤣... Onfray victime du #Covid ...
RT @780AM: #COVID🦠| "En 24 horas del inicio de los síntomas, la persona ya se puede hisopar. Muchos esperan hasta cinco días y ya le contag…
RT @RatnaK_Nanduri: #OxygenBeds available at #Guntur contact: Community Health Centre +91 86442 46941 #Validated on 24thMay 11:10 AM IST #C…
#オードリー・タン デジタルとAIの未来を語る
インタビュー時間20時間超!
#テクノロジー界 の叡智が、描く
デジタルとAI(人工知能)の未来!
#台湾 は、2020年に全世界を襲